In [1]:
# Third party packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import display

# Adding path
from os.path import abspath
import sys
sys.path.insert(0, abspath('..'))

# 
from query_request import *
from query_def import *
import credentials
# from sqlalchemy import create_engine

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth=None


In [2]:
# Demo for LDM

# Upload dataset
# downlaod dataset

In [3]:
# Demo for dragoman

# [Creating Endpoint](../configs/docker_command_icews.sh)

In [4]:
# create rdfmts
from DeTrusty import run_query
from DeTrusty.Molecule.MTCreation import create_rdfmts
from DeTrusty.Molecule.MTManager import ConfigFile

# Creating endpoints dictionary
endpoints = dictfrmjson('../configs/endpoints_sample.json')
print(endpoints)

# Creating RDF molecule template (rdfmts) from endpoints
# log_rdfmts = create_rdfmts(endpoints, '../configs/rdfmts.json')

# if already rdfmts has been created 
config_private = ConfigFile('../configs/rdfmts_private.json')
config_public = ConfigFile('../configs/rdfmts_public.json')

Total time taken by dictfrmjson function: 5.459785461425781e-05
{'https://labs.tib.eu/sdm/worldbank_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/icews_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/lei_endpoint/sparql': {}, 'https://skynet.coypu.org/coypu-internal': {'username': 'Your UserName', 'password': 'Your Password'}}


## Endpoints Status

In [5]:
# Demo query to all available enpoints

# demo query
query_test = """SELECT * WHERE{ ?Subject a ?Concept }LIMIT 1000"""


# creating request for cmemc
display("############### Cmemc Endpoint ###############")
cmemc_request = SPARQLRequest(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
# executing query and printing response
cmemc_request.execute(query_test)
buf = StringIO(cmemc_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for skynet
display("############### Skynet Endpoint ###############")
skynet_request = SPARQLRequest(skynet_endpoint, skynet_user, skynet_pass, 'basic')
# executing query and printing response
skynet_request.execute(query_test)
buf = StringIO(skynet_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for worldbank
display("############### Worldbank Endpoint ###############")
worldbank_request = SPARQLRequest(worldbank_endpoint)
# executing query and printing response
worldbank_request.execute(query_test)
display(worldbank_request.response.status_code)
buf = StringIO(worldbank_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for LEI
display("############### LEI Endpoint ###############")
lei_request = SPARQLRequest(lei_endpoint)
# executing query and printing response
lei_request.execute(query_test)
buf = StringIO(lei_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for ICEWS
display("############### ICEWS Endpoint ###############")
icews_request = SPARQLRequest(icews_endpoint)
# executing query and printing response
icews_request.execute(query_test)
buf = StringIO(icews_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))


'############### Cmemc Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.7718029022216797


,Subject,Concept
995,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/2000/01/rdf-schema#Resource
996,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/ns/shacl#PropertyShape
997,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
998,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2000/01/rdf-schema#Resource
999,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2002/07/owl#DatatypeProperty


'############### Skynet Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.16899585723876953


...

Passed Query Status: 200
Total time taken by execute function: 0.15626287460327148


,Subject,Concept
995,https://data.coypu.org/lei/2138006HHBEMF1TGWU29-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
996,https://data.coypu.org/lei/2138006HHUG8LC4UD387-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
997,https://data.coypu.org/lei/2138006HHXWLEK5WVK02-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
998,https://data.coypu.org/lei/2138006HI1P4JTM2WM22-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
999,https://data.coypu.org/lei/2138006HI5AF5E1P4T95-LA,https://www.gleif.org/ontology/Base/PhysicalAddress


'############### ICEWS Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.15753841400146484


,Subject,Concept
995,https://data.coypu.org/icews/36616020,https://schema.coypu.org/global#Conflict
996,https://data.coypu.org/icews/36616021,https://schema.coypu.org/global#Conflict
997,https://data.coypu.org/icews/36616024,https://schema.coypu.org/global#Conflict
998,https://data.coypu.org/icews/36616025,https://schema.coypu.org/global#Conflict
999,https://data.coypu.org/icews/36616026,https://schema.coypu.org/global#Conflict


## Federated Query

In [6]:
# creating request for Federated Query
display("############### Federated Query ###############")
fdq_request = SPARQLRequest(is_fdq=True)
fdq_request.execute(query_test, config_public, False)
fdq_request.save('../query_results/', 'query_test')

'############### Federated Query ###############'

2022-12-03 00:25:09,582 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql
2022-12-03 00:25:09,588 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://dbpedia.org/sparql
2022-12-03 00:25:09,591 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://query.wikidata.org/sparql
2022-12-03 00:25:09,604 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/lei_endpoint/sparql/
2022-12-03 00:25:09,611 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/icews_endpoint/sparql/


Total time taken by execute function: 1.3112115859985352
Index                    128
Subject.value           8000
Subject.type            8000
Concept.value           8000
Concept.type            8000
__meta__.is_verified    1000
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Subject.value         1000 non-null   object
 1   Subject.type          1000 non-null   object
 2   Concept.value         1000 non-null   object
 3   Concept.type          1000 non-null   object
 4   __meta__.is_verified  1000 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 32.4+ KB
None
Total time taken by json_to_csv function: 0.036043405532836914
Total time taken by save function: 0.036505937576293945


2022-12-03 00:29:13,636 - DeTrusty.Wrapper.RDFWrapper - ERROR - Exception while sending request to https://dbpedia.org/sparql - msg: <urlopen error [Errno 101] Network is unreachable> - query: SELECT ?Subject ?Concept WHERE {
        ?Subject <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?Concept

} LIMIT 10000 OFFSET 1350000


In [ ]:
print(query_4_fdq_ex_1)
query_4_fdq_ex_1 = """PREFIX coy: <https://schema.coypu.org/global#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX time: <http://www.w3.org/2006/time#>

PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wds: <http://www.wikidata.org/entity/statement/>
PREFIX wdv: <http://www.wikidata.org/value/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?year (COUNT(?iri) AS ?no_of_events) 
WHERE {
    ?iri a coy:Conflict .
    ?iri coy:hasIsoCode 'IDN' .
    ?iri coy:hasTimestamp ?timestamp .
    ?iri coy:hasFatalities ?fatalities .
    BIND(year(?timestamp) as ?year)
    
    ?countryWiki p:P298 ?isoCode .
    ?isoCode ps:P298 'IDN' .
    ?countryWiki p:P1082 ?itemP .
    ?itemP pq:P585 ?time .
    ?itemP ps:P1082 ?population .
    BIND(year(?time) as ?year)
}
GROUP BY ?year"""

fdq_request.execute(query_4_fdq_ex_1, config_private, False)
fdq_request.save('../query_results/', 'query_4_fdq_ex_1')


PREFIX coy: <https://schema.coypu.org/global#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX time: <http://www.w3.org/2006/time#>

PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wds: <http://www.wikidata.org/entity/statement/>
PREFIX wdv: <http://www.wikidata.org/value/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?isoCode ?year (COUNT(?iri) AS ?no_of_events) 
WHERE {
    ?iri a coy:AcledEvent .
    ?iri coy:hasIsoCode 'IDN' .
    ?iri coy:hasTimestamp ?timestamp .
    ?iri coy:hasFatalities ?fatalities .
    BIND(year(?timestamp) as ?year)

    ?countryWiki p:P298 ?isoCode .
  

Process Process-3:1:2:
Traceback (most recent call last):
  File "/home/tibvyasa/miniconda3/envs/coypu_demo/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/tibvyasa/miniconda3/envs/coypu_demo/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tibvyasa/miniconda3/envs/coypu_demo/lib/python3.10/site-packages/DeTrusty/Operators/AnapsidOperators/Xbind.py", line 70, in execute
    b_value = self.bind.term
Process Process-3:1:1:2:
Traceback (most recent call last):
  File "/home/tibvyasa/miniconda3/envs/coypu_demo/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
AttributeError: 'Bind' object has no attribute 'term'
  File "/home/tibvyasa/miniconda3/envs/coypu_demo/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tibvyasa/miniconda3/envs/coypu_demo/lib/python3.10/site-pac

In [ ]:

df = pd.read_csv('../../query_results/query_4_fdq_ex_1.csv', encoding='utf-8')

df